A hydrostatic, inviscid, adiabatic, and dry system in isentropic coordinates
==

In [ ]:
from datetime import timedelta
import gridtools as gt
import numpy as np
import os
import tasmania as taz

# For interactive plotting
from IPython import display
%matplotlib nbagg

import namelist_isentropic_dry as nl

Pre-processing
--

In [ ]:
# Create the underlying grid
grid = taz.GridXYZ(nl.domain_x, nl.nx, nl.domain_y, nl.ny, nl.domain_z, nl.nz,
                   topo_type=nl.topo_type, topo_time=nl.topo_time, topo_kwargs=nl.topo_kwargs,
                   dtype=nl.dtype)

# Instantiate the initial state
if nl.isothermal:
    state = taz.get_isothermal_isentropic_state(grid, nl.init_time,
                                                nl.init_x_velocity, nl.init_y_velocity,
                                                nl.init_temperature, dtype=nl.dtype)
else:
    state = taz.get_default_isentropic_state(grid, nl.init_time,
                                             nl.init_x_velocity, nl.init_y_velocity,
                                             nl.init_brunt_vaisala, dtype=nl.dtype)

# Instantiate the component inferring the diagnostic variables
pt = state['air_pressure_on_interface_levels'][0, 0, 0]
dv = taz.IsentropicDiagnostics(grid, moist_on=False, pt=pt,
                               backend=nl.backend, dtype=nl.dtype)

# Instantiate the component calculating the pressure gradient in isentropic coordinates
order = 4 if nl.horizontal_flux_scheme == 'fifth_order_upwind' else 2
pg = taz.ConservativeIsentropicPressureGradient(grid, order=order,
                                                horizontal_boundary_type=nl.horizontal_boundary_type, 
                                                backend=nl.backend, dtype=nl.dtype)

# Instantiate the component retrieving the velocity components
vc = taz.IsentropicVelocityComponents(grid, horizontal_boundary_type=nl.horizontal_boundary_type,
                                      reference_state=state, backend=nl.backend, dtype=nl.dtype)

# Wrap the physical components in a SequentialUpdateSplitting object
sus = taz.SequentialUpdateSplitting(dv, pg, vc, time_integration_scheme='forward_euler',
                                    grid=grid, horizontal_boundary_type=None)

# Instatiate the dynamical core
dycore = taz.HomogeneousIsentropicDynamicalCore(grid, moist_on=False,
                                                time_integration_scheme=nl.time_integration_scheme,
                                                horizontal_flux_scheme=nl.horizontal_flux_scheme,
                                                horizontal_boundary_type=nl.horizontal_boundary_type,
                                                damp_on=nl.damp_on, damp_type=nl.damp_type,
                                                damp_depth=nl.damp_depth, damp_max=nl.damp_max,
                                                damp_at_every_stage=nl.damp_at_every_stage,
                                                smooth_on=nl.smooth_on, smooth_type=nl.smooth_type,
                                                smooth_coeff=nl.smooth_coeff,
                                                smooth_at_every_stage=nl.smooth_at_every_stage,
                                                backend=nl.backend, dtype=nl.dtype)

# The drawers and the artist generating the left subplot
drawer1_properties = {
    'fontsize': 16, 'cmap_name': 'BuRd', 'cbar_on': True, 
    'cbar_levels': 18, 'cbar_ticks_step': 4, 'cbar_center': 15,
    'cbar_orientation': 'horizontal',
    'cbar_x_label': 'Horizontal velocity [m s$^{-1}$]',
    'draw_vertical_levels': False,
    'alpha': 0.5,
}
drawer1 = taz.Contourf(
    grid, 'horizontal_velocity', 'm s^-1', z=-1,  
    xaxis_units='km', yaxis_units='km', properties=drawer1_properties,
)
drawer2_properties = {
    'fontsize': 16, 'x_step': 2, 'y_step': 2, 'colors': 'black',
    'draw_vertical_levels': False, 'alpha': 0.5,
}
drawer2 = taz.Quiver(
    grid, z=-1, xaxis_units='km', yaxis_units='km',
    xcomp_name='x_velocity', xcomp_units='m s^-1',
    ycomp_name='y_velocity', ycomp_units='m s^-1',
    properties=drawer2_properties
)
axes1_properties = {
    'fontsize': 16, 'title_left': '$\\theta = 300$ K',
    'x_label': '$x$ [km]', 'x_lim': [-250, 250],
    'y_label': '$y$ [km]', 'y_lim': [-250, 250],
}
plot1 = taz.Plot((drawer1, drawer2), axes_properties=axes1_properties)

# The drawer and the artist generating the right subplot
drawer3_properties = {
    'fontsize': 16, 'cmap_name': 'BuRd', 'cbar_on': True, 
    'cbar_levels': 18, 'cbar_ticks_step': 4, 'cbar_center': 15,
    'cbar_orientation': 'horizontal',
    'cbar_x_label': '$x$-velocity [m s$^{-1}$]',
    'draw_vertical_levels': True,
    'linecolor': 'black',
}
drawer3 = taz.Contourf(
    grid, 'x_velocity_at_u_locations', 'm s^-1', y=25,  
    xaxis_units='km', zaxis_name='height', zaxis_units='km',
    properties=drawer3_properties,
)
axes3_properties = {
    'fontsize': 16, 'title_left': '$y = 0$ km',
    'x_label': '$x$ [km]', 'x_lim': [-250, 250],
    'y_label': '$z$ [km]', 'y_lim': [0, 14],
}
plot2 = taz.Plot(drawer3, axes_properties=axes3_properties)

# The monitor encompassing and coordinating the two artists
figure_properties = {'fontsize': 16, 'figsize': (12, 7), 'tight_layout': True}
plot_monitor = taz.PlotComposite(
    nrows=1, ncols=2, artists=(plot1, plot2), interactive=False, figure_properties=figure_properties
)

# Create a monitor to dump to the solution into a NetCDF file
if nl.filename is not None:
    if os.path.exists(nl.filename):
        os.remove(nl.filename)
    netcdf_monitor = taz.NetCDFMonitor(nl.filename, grid)
    netcdf_monitor.store(state)

Simulation
--

In [ ]:
# Simulation settings
dt = nl.timestep
nt = nl.niter
 
# Integrate
for i in range(nt):
    # Update the (time-dependent) topography
    dycore.update_topography((i+1) * dt)
  
    # Compute the dynamics
    state_new = dycore(state, {}, dt)
    state.update(state_new)
    
    # Compute the physics, and couple it with the dynamics
    _ = sus(state=state, timestep=dt)
    
    if (nl.print_frequency > 0) and ((i + 1) % nl.print_frequency == 0):
        u = state['x_velocity_at_u_locations'].to_units('m s^-1').values[...]
        v = state['y_velocity_at_v_locations'].to_units('m s^-1').values[...]
  
        umax, umin = u.max(), u.min()
        vmax, vmin = v.max(), v.min()
        cfl = max(umax * dt.total_seconds() / grid.dx.to_units('m').values.item(),
                  vmax * dt.total_seconds() / grid.dy.to_units('m').values.item())
 
        # Print useful info
        print('Iteration {:6d}: CFL = {:4f}, umax = {:8.4f} m/s, umin = {:8.4f} m/s, '
              'vmax = {:8.4f} m/s, vmin = {:8.4f} m/s'.format(i+1, cfl, umax, umin, vmax, vmin))
 
    if (nl.filename is not None) and (((nl.save_frequency > 0) and ((i + 1) % nl.save_frequency == 0)) or i + 1 == nt):
        # Save the solution
        netcdf_monitor.store(state)

    if (nl.plot_frequency > 0) and ((i+1) % nl.plot_frequency == 0):
        # Plot the solution
        plot1.axes_properties['title_right'] = str((i+1)*dt)
        plot2.axes_properties['title_right'] = str((i+1)*dt)
        fig = plot_monitor.store(((state, state), state), show=False)
        display.clear_output(wait=True)
        display.display(fig)
        
print('Simulation successfully completed. HOORAY!')

Post-processing
--

In [ ]:
if nl.filename is not None:
    netcdf_monitor.write()